In [ ]:
import logging
from stanfordcorenlp import StanfordCoreNLP
import numpy as np
import re

model_path = "/home/qtxu/PLM/stanford-corenlp-full-2018-02-27"
zh_model = StanfordCoreNLP(model_path, lang='zh', quiet=True, logging_level=logging.DEBUG)

sen = "这款车的发动机异于同类。"
words = list(zh_model.word_tokenize(sen))
print(words)
print("该句子解析后的长度是：",len(words))
arcs = list(zh_model.dependency_parse(sen))
print(arcs) # include Root

arcs.sort(key=lambda x: x[2]) # 按照在sentence中出现的次序排序
print(arcs)
words = [w + "-" + str(idx) for idx, w in enumerate(words)]
rely_id = [arc[1] for arc in arcs]
print("rely_id", rely_id)
relation = [arc[0] for arc in arcs]
print("relation", relation)
# heads = ['Root' if id == 0 else words[id - 1] for id in rely_id]
heads = ['Root' if id == 0 else words[id - 1] for id in rely_id]
matrix1 = np.zeros((len(words), len(words))).astype('float32')

def dependency_adj_matrix_zh(sen, sent_tokened):
    stanford_words = list(zh_model.word_tokenize(sen))
    # sen_tokens_text_list = tokenizer.tokenize(text)
    # nlp = spacy.load("zh_core_web_sm") # 加载spaCy模型
    # doc = nlp(text) # 解析文本

    # spacy_tokens = []
    # for token in doc:
    #     spacy_tokens.append(token) # ['油耗', '比', '骑车', '要', '高', '。']
    # print(spacy_tokens)

    # 依据spacy的分词解析结果，存放开始的index
    test_dict = {}
    count = 0
    for word in stanford_words:
        # print(type(word.text))
        if word.like_num:
            test_dict[count] = word
            count += 1
        elif word.text == "DX":
            test_dict[count] = word
            count += 1  
        elif len(re.findall(r"\d+[a-zA-Z]+[\u4e00-\u9fff]+", word.text)) != 0: # 匹配数字字母汉字格式的str, eg 4s站
            test_dict[count] = word
            count += 1
        elif len(re.findall(r"\d+[\u4e00-\u9fff]+", word.text)) != 0: # 匹配数字汉字格式， eg:
            test_dict[count] = word
            count +=1
        elif len(re.findall(r"[a-zA-Z]+\d+", word.text)) != 0:  # 匹配字母数字格式， eg:5W
            test_dict[count] = word
            count += 1
        elif len(re.findall(r"[\u4e00-\u9fff]+\d", word.text)) != 0: #匹配汉字数字格式，eg:
            test_dict[count] = word
            count += 1   
        elif re.findall(r"[a-zA-Z]", word.text):
            test_dict[count] = word
            count += 1         
        else: # 正常的情况
            test_dict[count] = word
            count+=len(word)
    # print("spacy results:", test_dict)
    # print(test_dict) # {0: '油耗', 2: '比', 3: '骑车', 5: '要', 6: '高', 7: '。'}

    number = 0
    adj_matrix = np.eye(len(sent_tokened)) # sent_tokened:表示的是经过berttokenfast分词之后的结果
    i = 0
    while i < len(sent_tokened):
        word = stanford_words[number]# 获取第number个stanford解析的结果

        word_sp = list(word.text)
        for child in word.children:
            adj_word_list = list(child.text) # 具有依存关系的词语
            word_list = list(word.text) # spacy分词的结果中的某一个词
            child_key = next(key for key, val in test_dict.items() if val == child) # obtain the start index of child
            word_key = next(key for key, val in test_dict.items() if val == word) # obtain the start index of spacy_word
            # print("child:{}, word:{}".format(child, word))
            for m in range(child_key, len(adj_word_list) + child_key):
                for n in range(word_key, len(word_list) + word_key):
                    try:
                        adj_matrix[m][n] = 1
                        adj_matrix[n][m] = 1
                    except:
                        print(text)
                        # stop()
        i += len(word_sp)
        number += 1
    return adj_matrix


for i in range(len(words)):
    print(relation[i] + '(' + words[i] + ', ' + heads[i] + ')')

: 

In [ ]:
import logging
from stanfordcorenlp import StanfordCoreNLP
import numpy as np
import re

model_path = "/home/qtxu/PLM/stanford-corenlp-full-2018-02-27"
zh_model = StanfordCoreNLP(model_path, lang='zh', quiet=True, logging_level=logging.DEBUG)

sen = "这款车的发动机异于同类。"
words = list(zh_model.word_tokenize(sen))
print(words)
print("该句子解析后的长度是：",len(words))
arcs = list(zh_model.dependency_parse(sen))
print(arcs) # include Root

: 

In [11]:
test_dict={0: '这', 1: '款', 2: '车', 3: '的', 4: '发动机', 7: '。'}
stan_dict={'这-0': '车-2', '款-1': '这-0', '车-2': '发动机-4', '的-3': '车-2', '发动机-4': 'Root', '。-5': '发动机-4'}
for key_i, value_i in stan_dict.items():
    # print("stan_dict", stan_dict)
    key_word, key_id = key_i.split("-")
    try:
        value_word, value_id = value_i.split("-")
    except:
        continue

    key_start = next(key for key, value in test_dict.items() if value == key_word)
    value_start = next(key for key, value in test_dict.items() if value == value_word)
